In [8]:
import tensorflow as tf

In [9]:
from tensorflow.examples.tutorials.mnist import input_data

In [10]:
mnist = input_data.read_data_sets("C:/Users/princ/Desktop/Tensorflow Bootcamp Master/03-Convolutional-Neural-Networks/MNIST_data/", one_hot=True)

Extracting C:/Users/princ/Desktop/Tensorflow Bootcamp Master/03-Convolutional-Neural-Networks/MNIST_data/train-images-idx3-ubyte.gz
Extracting C:/Users/princ/Desktop/Tensorflow Bootcamp Master/03-Convolutional-Neural-Networks/MNIST_data/train-labels-idx1-ubyte.gz
Extracting C:/Users/princ/Desktop/Tensorflow Bootcamp Master/03-Convolutional-Neural-Networks/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting C:/Users/princ/Desktop/Tensorflow Bootcamp Master/03-Convolutional-Neural-Networks/MNIST_data/t10k-labels-idx1-ubyte.gz


In [11]:
#Helper


#Initialize the Weights
def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(init_random_dist)

#Initialize Bias
def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

#Conv2D
def conv2d(x,W):
    # x --> Input tensor [Batch,Height,Width,Channels(RGB)]
    # W --> Kernel filter [filter Height, filter Width, Number of Input Images, Number of Output features(depth)]
    
    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

#Pooling
def max_pool_2by2(x):
    # x --> Input tensor [Batch,Height,Width,Channels(RGB)]
    
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
                            #ksize is the size of the square we are sliding across the image
                            #strides is the number of pixels that square is sliding along one axis of the image
                            #here, we have a ksize of 2by2 and a stride of 2 (the 1s are for the 1 image and 1 channel)

In [12]:
# Convolution ReLu Layer 
def convolutional_layer(input_x,shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

In [16]:
# Normal FC (Fully Connected)
def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b= init_bias([size])
    return tf.matmul(input_layer,W) + b

In [15]:
with tf.Session() as sess:
    test = tf.constant(0.1, shape=[5])
    run = sess.run(test)
    print(run)

[ 0.1  0.1  0.1  0.1  0.1]


In [20]:
#Placeholders:
x = tf.placeholder(tf.float32, shape=[None, 784])
hold_prob = tf.placeholder(tf.float32)
y_true = tf.placeholder(tf.float32, shape = [None, 10])

In [21]:
#Layers
x_image = tf.reshape(x,[-1,28,28,1]) #[Batch,Height,Width,Channels(RGB)]
convo_1 = convolutional_layer(x_image, shape=[5,5,1,32]) #[filter Height, filter Width, Number of Channels IN, Number of features(depth)]
convo_1_pooling = max_pool_2by2(convo_1)
convo_2 = convolutional_layer(convo_1_pooling,shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)
convo_2_flat = tf.reshape(convo_2_pooling,[-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat,1024))
full_one_dropout = tf.nn.dropout(full_layer_one,keep_prob = hold_prob)
y_pred = normal_full_layer(full_one_dropout, 10)

In [22]:
# Loss Function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_true,logits=y_pred))

In [23]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
train = optimizer.minimize(cross_entropy)

In [24]:
#initialize variables:
init = tf.global_variables_initializer()

In [27]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        batch_x, batch_y = mnist.train.next_batch(50)
        sess.run(train,feed_dict={x: batch_x, y_true: batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            print("On Step : {}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1),tf.argmax(y_true,1))
            acc = tf.reduce_mean(tf.cast(matches,tf.float32))
            print(sess.run(acc,feed_dict={x:mnist.test.images,y_true:mnist.test.labels,hold_prob:1.0}))

On Step : 0
Accuracy: 
0.188
On Step : 100
Accuracy: 
0.9482
On Step : 200
Accuracy: 
0.9682
On Step : 300
Accuracy: 
0.9717
On Step : 400
Accuracy: 
0.9709
On Step : 500
Accuracy: 
0.9759
On Step : 600
Accuracy: 
0.9774
On Step : 700
Accuracy: 
0.978
On Step : 800
Accuracy: 
0.9821
On Step : 900
Accuracy: 
0.9829


KeyboardInterrupt: 

# NOTE THAT I HAVE VOLONTARIYL CAUSED THIS ERROR BY STOPPING THE PROGRAM!! BECAUSE IT WOULD TAKE A WHOLE LOT OF TIME TO GO THROUGH 5000 STEPS... 

# CURRENT STATE OF THE CNN: ACCURACY OF 0.9829